In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm

# import cupy as cp
# import cudf
# import cuml
# from cuml.preprocessing import OneHotEncoder
# from cuml.preprocessing import MinMaxScaler
# from cuml import train_test_split as cutts

# import dask_ml
# import dask_ml.model_selection as dcv
# from dask_ml.preprocessing import DummyEncoder
# from dask_ml.preprocessing import Categorizer
# from dask_ml.preprocessing import OneHotEncoder
# from dask_ml.preprocessing import MinMaxScaler

import xgboost

import copy

import itertools

import re

import import_ipynb

from _Self_Written_Functions_Sheet import (conditions_edit, nbrhd_rank, drop_nbrhd,
totallivsf_add, drop_grlivarea, bsmtfin_add, drop_bsmtfinsf12, totalporchsf_add,
drop_porches, yrsremodtosold_add, drop_yrsoldremod, fireplace_yes, drop_fireplaces,
pool_yes, drop_poolarea, totalbaths_add, drop_baths, totalbsmtbaths_add,
drop_bsmtbaths)

from _Self_Written_Functions_Sheet import (mlr_model, lasso_model, ridge_model, enet_model, 
                                           rfr_model, gbm_model, xgb_model)

from _Self_Written_Functions_Sheet import (dum_scale, use_og_data, comb_list_generator,
                                           apply_comb_list, rmse)

importing Jupyter notebook from _Self_Written_Functions_Sheet.ipynb


In [2]:
# importing the datasets
# hp_train = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_train.csv', index_col=0)
# hp_test = pd.read_csv('Proj3_Datasets/hp_2a_ranked_edited_test.csv', index_col=0)
# hp_logsp = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_logsaleprice.csv',
#                                 index_col=0)
# hp_saleprice = pd.read_csv('Proj3_Datasets/hp_1a_no_imputation_saleprice.csv')

hp_train = pd.read_csv('..\hp_2a_ranked_edited_train.csv', index_col=0)
hp_test = pd.read_csv('..\hp_2a_ranked_edited_test.csv', index_col=0)
hp_logsp = pd.read_csv('..\hp_1a_no_imputation_logsaleprice.csv', index_col=0)
hp_saleprice = pd.read_csv('..\hp_1a_no_imputation_saleprice.csv', index_col=0)

# saving train index
hp_index = hp_train.index

combo_nonedit = pd.concat([hp_train, hp_test])
combo_nonedit.columns = hp_train.columns

# convert all possible numeric types to numeric
for col in combo_nonedit.columns: 
    combo_nonedit[col] = combo_nonedit[col].astype('float', errors='ignore')

combo_cols = list(combo_nonedit.columns)

combo = copy.deepcopy(combo_nonedit)

In [3]:
combo.loc[hp_index]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,2-STORY 1946 & NEWER,RL,65.0,8450.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4.0,3.0,PConc,4.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,GasA,5.0,1.0,5.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,8.0,7.0,0.0,None,Attchd,2003.0,2.0,2.0,548.0,3.0,3.0,3.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2008.0,WD,Normal
1,1-STORY 1946 & NEWER ALL STYLES,RL,80.0,9600.0,1.0,0.0,Reg,Lvl,3.0,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6.0,8.0,1976.0,1976.0,Gable,CompShg,MetalSd,MetalSd,None,0.0,3.0,3.0,CBlock,4.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,GasA,5.0,1.0,5.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,3.0,6.0,7.0,1.0,TA,Attchd,1976.0,2.0,2.0,460.0,3.0,3.0,3.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,5.0,2007.0,WD,Normal
2,2-STORY 1946 & NEWER,RL,68.0,11250.0,1.0,0.0,IR1,Lvl,3.0,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2001.0,2002.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4.0,3.0,PConc,4.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,GasA,5.0,1.0,5.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,4.0,6.0,7.0,1.0,TA,Attchd,2001.0,2.0,2.0,608.0,3.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,9.0,2008.0,WD,Normal
3,2-STORY 1945 & OLDER,RL,60.0,9550.0,1.0,0.0,IR1,Lvl,3.0,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7.0,5.0,1915.0,1970.0,Gable,CompShg,Wd Sdng,WdShing,None,0.0,3.0,3.0,BrkTil,3.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,GasA,4.0,1.0,5.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,4.0,7.0,7.0,1.0,Gd,Detchd,1998.0,1.0,3.0,642.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,None,0.0,2.0,2006.0,WD,Abnorml
4,2-STORY 1946 & NEWER,RL,84.0,14260.0,1.0,0.0,IR1,Lvl,3.0,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8.0,5.0,2000.0,2000.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,4.0,3.0,PConc,4.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,GasA,5.0,1.0,5.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,4.0,9.0,7.0,1.0,TA,Attchd,2000.0,2.0,3.0,836.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,12.0,2008.0,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,2-STORY 1946 & NEWER,RL,62.0,7917.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6.0,5.0,1999.0,2000.0,Gable,CompShg,VinylSd,VinylSd,None,0.0,3.0,3.0,PConc,4.0,3.0,1.0,1.0,0.0,1.0,0.0,953.0,953.0,GasA,5.0,1.0,5.0,953.0,694.0,0.0,1647.0,0.0,0.0,2.0,1.0,3.0,1.0,3.0,7.0,7.0,1.0,TA,Attchd,1999.0,2.0,2.0,460.0,3.0,3.0,3.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,8.0,2007.0,WD,Normal
1454,1-STORY 1946 & NEWER ALL STYLES,RL,85.0,13175.0,1.0,0.0,Reg,Lvl,3.0,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6.0,6.0,1978.0,1988.0,Gable,CompShg,Plywood,Plywood,Stone,119.0,3.0,3.0,CBlock,4.0,3.0,1.0,5.0,790.0,3.0,163.0,589.0,1542.0,GasA,3.0,1.0,5.0,2073.0,0.0,0.0,2073.0,1.0,0.0,2.0,0.0,3.0,1.0,3.0,7.0,6.0,2.0,TA,Attchd,1978.0,1.0,2.0,500.0,3.0,3.0,3.0,349.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
def mlr_model(x_trn, y_trn, x_tst, y_tst):
    lm = LinearRegression()
    # parameters to use
    fitintercept = [True, False]
    gparam_mlr_1 = {'fit_intercept': fitintercept}
    # setting parameters
    gs_mlr_1 = ms.GridSearchCV(lm, gparam_mlr_1, cv=n_folds, refit=True, n_jobs=-1,
                               scoring='neg_root_mean_squared_error',
                               return_train_score=True)
    gs_mlr_1.fit(x_trn, y_trn)
    mlr_model.test_rmse = rmse(gs_mlr_1, y_tst, x_tst)    
    
def lasso_model(x_trn, y_trn, x_tst, y_tst):
    lasso = Lasso(max_iter=1000000)
    # parameters to use
    alphas = [0.0001, 0.000112, 0.000124, 0.000136, 0.000148]
    fitintercept = [True, False]
    selec = ['cyclic', 'random']
    gparam_lasso_1 = {'alpha': alphas,
                      'fit_intercept':fitintercept,
                      'selection': selec}
    # setting parameters
    gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                                 scoring='neg_root_mean_squared_error',
                                 return_train_score=True)
    gs_lasso_1.fit(x_trn, y_trn)
    lasso_model.test_rmse = rmse(gs_lasso_1, y_tst, x_tst)

# def ridge_model(x_trn, y_trn, x_tst, y_tst):
#     ridge = Ridge(random_state=state, max_iter=10000, fit_intercept=True)
#     # parameters to use
#     solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
#     fitintercept = [True, False]
#     gparam_ridge_1 = {'solver': solvers,
#                       'fit_intercept': fitintercept, 
#                       'random_state': [state]}
#     # setting parameters
#     gs_ridge_1 = ms.GridSearchCV(ridge, gparam_ridge_1, cv=n_folds, refit=True, n_jobs=-1,
#                                  scoring='neg_root_mean_squared_error',
#                                  return_train_score=True)
#     gs_ridge_1.fit(x_trn, y_trn)
#     ridge_model.test_rmse = rmse(gs_ridge_1, y_tst, x_tst)
    
def enet_model(x_trn, y_trn, x_tst, y_tst):
    enet = ElasticNet(max_iter=10000000, selection='random')
    # parameters to use
    alphas = [1e-4, 2.5e-4, 5e-4]
    l1ratio = [0.5, 0.7, 1]
    precomputes = [True, False]
    warmstart = [True, False]
    fitintercept = [True, False]
    gparam_enet_1 = {'alpha': alphas,
                     'l1_ratio': l1ratio,
                     'precompute': precomputes, 
                     'warm_start': warmstart,
                     'fit_intercept': fitintercept}
    # setting parameters
    gs_enet_1 = ms.GridSearchCV(enet, gparam_enet_1, cv=n_folds, refit=True, n_jobs=-1,
                                scoring='neg_root_mean_squared_error', 
                                return_train_score=True)
    gs_enet_1.fit(x_trn, y_trn)
    enet_model.test_rmse = rmse(gs_enet_1, y_tst, x_tst)
    
def rfr_model(x_trn, y_trn, x_tst, y_tst):
    rfr = RandomForestRegressor()
    gparam_rfr = {}
    # setting parameters
    gs_rfr = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                             scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_rfr.fit(x_trn, y_trn)
    rfr_model.test_rmse = rmse(gs_rfr, y_tst, x_tst)
    
def gbm_model(x_trn, y_trn, x_tst, y_tst):
    gbm = GradientBoostingRegressor()
    gparam_gbm = {}
    gs_gbm = ms.GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_gbm.fit(x_trn, y_trn)
    gbm_model.test_rmse = rmse(gs_gbm, y_tst, x_tst)
    
def xgb_model(x_trn, y_trn, x_tst, y_tst):
    xgb_t = xgboost.XGBRegressor()
    gparam_xgb_t = {}
    gs_xgb_t = ms.GridSearchCV(xgb_t, gparam_xgb_t, cv=n_folds, refit=True, n_jobs=-1,
                            scoring='neg_root_mean_squared_error', return_train_score=True)
    gs_xgb_t.fit(x_trn, y_trn)
    xgb_model.test_rmse = rmse(gs_xgb_t, y_tst, x_tst)

In [5]:
hp = copy.deepcopy(combo_nonedit)
state = 0
counter = 0

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# lists of the functions separated by whether something is changed or added
stuff_add = [nbrhd_rank, totallivsf_add, bsmtfin_add, totalporchsf_add, 
             yrsremodtosold_add, fireplace_yes, pool_yes, totalbaths_add,
             totalbsmtbaths_add, conditions_edit]

# list of functions that will drop columns
stuff_drop = [drop_nbrhd, drop_grlivarea, drop_bsmtfinsf12,  drop_porches,
              drop_yrsoldremod, drop_fireplaces, drop_poolarea,
              drop_baths, drop_bsmtbaths]

# list of models to use
models = [mlr_model, lasso_model,  enet_model, rfr_model, gbm_model, xgb_model]

# using self-written function to generate permutations of 2 powersets
# looking at the max length
len(comb_list_generator(stuff_add, stuff_drop))

92378

In [6]:
# # will not be using a function for ease of access for variables

# state = 0
# counter = 0

# # numbers will be indices later down
# funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

# for cb_list in comb_list_generator(stuff_add, stuff_drop):
    
#     df = copy.deepcopy(combo_nonedit)

#     # converting any numbers to numeric type
#     for col in df.columns: 
#         df[col] = df[col].astype('float', errors='ignore')

#     # converting the single string within the list to a list of strings
#     cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
#     # converting the list of strings to a single string
#     funcs_used = ', '.join(cb_split)

#     # calling all of the functions within the list
#     for cb in cb_split:
#         eval(cb)(df)
        
#     # getting the train dataset for modeling after dummification
#     hp_touse = dum_scale(df).loc[hp_index]


#     # setting up train and test sets
#     xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
#                                                        test_size=0.2, 
#                                                        random_state=state)
#     # Must flatten to fit
#     ytrain = ytrain.values.flatten()

#     # run the models
#     for i, model in enumerate(models):
#         model(xtrain, ytrain, xtest, ytest)
#         # each model has a designated number (the order in the list: models)
#         i+=1
#         # add scores to the dictionary of dictionaries
#         funcs_scores[i][funcs_used] = model.test_rmse

#     # keeping track of rounds
#     counter += 1
#     if counter % 5 == 0:
#         print(f'Completed round {counter}')
    
# # return funcs_scores

In [7]:
# for k_1 in funcs_scores:
#     k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
#     v_min = funcs_scores[k_1][k_min]
#     print(k_1)
#     print(k_min)
#     print(v_min) 
#     print('#' * 50)

In [8]:
# for k_1 in funcs_scores:
#     k_min = min(funcs_scores[k_1], key=funcs_scores[k_1].get)
#     v_min = funcs_scores[k_1][k_min]
#     print(k_1)
#     print(k_min)
#     print(v_min) 
#     print('#' * 50)

In [9]:
# will not be using a function for ease of access for variables

state = 0
counter = 0
df_list = []
i = 0
# numbers will be indices later down
funcs_scores = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}}

for cb_list in comb_list_generator(stuff_add, stuff_drop):
    
    df = copy.deepcopy(combo_nonedit)

    # converting any numbers to numeric type
    for col in df.columns: 
        df[col] = df[col].astype('float', errors='ignore')

    # converting the single string within the list to a list of strings
    cb_split = re.sub("['']", "", str(cb_list)).strip('][').split(', ')
    # converting the list of strings to a single string
    funcs_used = ', '.join(cb_split)
    df_name = '__'.join(cb_split)

    # calling all of the functions within the list
    for cb in cb_split:
        eval(cb)(df)
        
    # getting the train dataset for modeling after dummification
#     hp_touse = dum_scale(df).loc[hp_index]
    
    locals()[df_name] = dum_scale(df).loc[hp_index]
    df_list.append(df_name)
    i+=1
    if i % 1000 == 0:
        print(i)

KeyboardInterrupt: 

In [10]:
df_list

['use_og_data',
 'nbrhd_rank__drop_nbrhd',
 'nbrhd_rank__drop_grlivarea',
 'nbrhd_rank__drop_bsmtfinsf12',
 'nbrhd_rank__drop_porches',
 'nbrhd_rank__drop_yrsoldremod',
 'nbrhd_rank__drop_fireplaces',
 'nbrhd_rank__drop_poolarea',
 'nbrhd_rank__drop_baths',
 'nbrhd_rank__drop_bsmtbaths',
 'totallivsf_add__drop_nbrhd',
 'totallivsf_add__drop_grlivarea',
 'totallivsf_add__drop_bsmtfinsf12',
 'totallivsf_add__drop_porches',
 'totallivsf_add__drop_yrsoldremod',
 'totallivsf_add__drop_fireplaces',
 'totallivsf_add__drop_poolarea',
 'totallivsf_add__drop_baths',
 'totallivsf_add__drop_bsmtbaths',
 'bsmtfin_add__drop_nbrhd',
 'bsmtfin_add__drop_grlivarea',
 'bsmtfin_add__drop_bsmtfinsf12',
 'bsmtfin_add__drop_porches',
 'bsmtfin_add__drop_yrsoldremod',
 'bsmtfin_add__drop_fireplaces',
 'bsmtfin_add__drop_poolarea',
 'bsmtfin_add__drop_baths',
 'bsmtfin_add__drop_bsmtbaths',
 'totalporchsf_add__drop_nbrhd',
 'totalporchsf_add__drop_grlivarea',
 'totalporchsf_add__drop_bsmtfinsf12',
 'totalporc

In [11]:
locals()[df_list[1]]

,LotFrontage,LotArea,Street,Alley,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,NbMedianRank,MSSubClass_1-1/2 STORY FINISHED ALL AGES,MSSubClass_1-1/2 STORY PUD - ALL AGES,MSSubClass_1-STORY 1945 & OLDER,MSSubClass_1-STORY 1946 & NEWER ALL STYLES,MSSubClass_1-STORY PUD (Planned Unit Development) - 1946 & NEWER,MSSubClass_1-STORY W/FINISHED ATTIC ALL AGES,MSSubClass_2 FAMILY CONVERSION - ALL STYLES AND AGES,MSSubClass_2-1/2 STORY ALL AGES,MSSubClass_2-STORY 1945 & OLDER,MSSubClass_2-STORY 1946 & NEWER,MSSubClass_2-STORY PUD - 1946 & NEWER,MSSubClass_DUPLEX - ALL STYLES AND AGES,MSSubClass_PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,MSSubClass_SPLIT FOYER,MSSubClass_SPLIT OR MULTI-LEVEL,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_TwnhsE,BldgType_TwnhsI,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_BrkComm,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_WdShing,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.150685,0.033420,1.0,0.0,1.0,0.666667,0.500,0.949275,0.883333,0.122500,0.666667,0.50,0.8,0.75,0.25,1.000000,0.176060,0.166667,0.000000,0.064212,0.168008,1.00,1.0,1.00,0.109641,0.413559,0.0,0.289015,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.461538,1.000000,0.00,0.996517,0.666667,0.4,0.368280,0.6,0.6,1.0,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.090909,0.50,0.458333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0

In [ ]:
for dfn in len(df_list)+1:
    locals()[dfn]


In [ ]:
for dfn in df_list:
    hp_touse = dfn.strip("'")
    
    # setting up train and test sets
    xtrain, xtest, ytrain, ytest = ms.train_test_split(hp_touse, hp_logsp,
                                                       test_size=0.2, 
                                                       random_state=state)
    # Must flatten to fit
    ytrain = ytrain.values.flatten()

    # run the models
    for i, model in enumerate(models):
        model(xtrain, ytrain, xtest, ytest)
        # each model has a designated number (the order in the list: models)
        i+=1
        # add scores to the dictionary of dictionaries
        funcs_scores[i][funcs_used] = model.test_rmse

    # keeping track of rounds
    counter += 1
    if counter % 5 == 0:
        print(f'Completed round {counter}')
    
# return funcs_scores